# Usage in custom model 

## Via Inheritance 

The most straightforward way to use `formulaic-contrasts` with a custom model is to use {class}`~formulaic_contrasts.FormulaicContrasts`
as a base class or mixin class.

As an example, let's wrap an Ordinary Least Squares ({class}`~statsmodels.regression.linear_model.OLS`) linear model into a custom class
for the use with `formulaic-contrasts`. The aim is to build a model that takes a pandas DataFrame and a formulaic formula as input
allows to fit the model to a continuous variable from the dataframe and perform a statistical test for a given contrast. 

This can be achived with the following class definition. The constructor, the {func}`~formulaic_contrasts.FormulaicContrasts.contrast` and {func}`~formulaic_contrasts.FormulaicContrasts.cond` methods are inherited from the {class}`~formulaic_contrasts.FormulaicContrasts`
base class:

In [1]:
import formulaic_contrasts
import numpy as np
import statsmodels.api as sm


class StatsmodelsOLS(formulaic_contrasts.FormulaicContrasts):
    def fit(self, variable: str):
        self.mod = sm.OLS(self.data[variable], self.design)
        self.mod = self.mod.fit()

    def t_test(self, contrast: np.ndarray):
        return self.mod.t_test(contrast)

Let's apply our model to an example dataset. The toy data mimicks a 2-arm clinical trial (`drugA` vs. `drugB`)
with Responders (`responder`) and Non-responders (`non_responder`) and a continuous biomarker that can protentially
predict response of the different treatments. 

In [2]:
df = formulaic_contrasts.datasets.treatment_response()
df

,treatment,response,biomarker
0,drugA,non_responder,6.595490
1,drugA,non_responder,7.071509
2,drugA,non_responder,8.537421
3,drugA,non_responder,6.787991
4,drugA,non_responder,10.109717
...,...,...,...
75,drugB,responder,11.167627
76,drugB,responder,9.493773
77,drugB,responder,5.027817
78,drugB,responder,9.800762


Let's fit the model an perform the statistical test

In [3]:
model = StatsmodelsOLS(df, "~ treatment * response")
model.fit("biomarker")
model.t_test(model.contrast("response", baseline="non_responder", group_to_compare="responder"))

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -1.6492      0.935     -1.764      0.082      -3.512       0.213

## Via Composition

Alternatively, if you prefer to work without inheritance, you can use `FormulaicContrast` as an attribute. 
In this case, you need to define {func}`~formulaic_contrasts.FormulaicContrasts.cond`/{func}`~formulaic_contrasts.FormulaicContrasts.contrast` yourself, or provide a custom way to define contrasts, calling {func}`~formulaic_contrasts.FormulaicContrasts.cond` internally. A minimal implementation could look like:

In [4]:
import pandas as pd


class StatsmodelsOLS:
    def __init__(self, data: pd.DataFrame, design: str) -> None:
        self.data = data
        self.contrast_builder = formulaic_contrasts.FormulaicContrasts(data, design)

    def fit(self, variable: str):
        self.mod = sm.OLS(self.data[variable], self.contrast_builder.design)
        self.mod = self.mod.fit()

    def cond(self, **kwargs):
        return self.contrast_builder.cond(**kwargs)

    def contrast(self, *args, **kwargs):
        return self.contrast_builder.contrast(*args, **kwargs)

    def t_test(self, contrast: np.ndarray):
        return self.mod.t_test(contrast)

The result is the same:

In [ ]:
model = StatsmodelsOLS(df, "~ treatment * response")
model.fit("biomarker")
model.t_test(model.contrast("response", baseline="non_responder", group_to_compare="responder"))

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -1.6492      0.935     -1.764      0.082      -3.512       0.213

## Manual usage

:::{hint}

Some helpful definitions for working with formulaic formulas (e.g. `~ 0 + C(donor):treatment + np.log1p(continuous)`):
 * A *term* refers to an expression in the formula, separated by `+`, e.g. `C(donor):treatment`, or `np.log1p(continuous)`.
 * A *variable* refers to a column of the data frame passed to formulaic, e.g. `donor`.
 * A *factor* is the specification of how a certain variable is represented in the design matrix, e.g. treatment coding with base level "A" and reduced rank.

:::

You can use the lower-level interface {func}`~formulaic_contrasts.get_factor_storage_and_materializer` to 
introspect formulaic models if the `FormulaicContrasts` class doesn't fit your needs.

`get_factor_storage_and_materializer` will generate a custom [materializer](https://matthewwardrop.github.io/formulaic/dev/extensions/) 
that, while generating the design matrix from the input data and formula, keeps track of certain metadata that are 
useful for defining contrasts.

In [6]:
factor_storage, variables_to_factors, materializer_class = formulaic_contrasts.get_factor_storage_and_materializer()

If we apply the materializer to the example dataset from above...

In [7]:
design_mat = materializer_class(df, record_factor_metadata=True).get_model_matrix("~ treatment * response")

... `factor_storage` will keep track of *factors* used in the formula, while `variables_to_factors` will keep 
track of *variables* used in the formula. The {class}`~formulaic_contrasts.FactorMetadata` objects contain information
that is useful for building custom contrasts, such as the list of categories or the {property}`~formulaic_contrasts.FactorMetadata.base` 
level of categorical variables. 

In [8]:
from pprint import pprint

pprint(factor_storage)

defaultdict(<class 'list'>,
            {'response': [FactorMetadata(name='response',
                                         reduced_rank=True,
                                         custom_encoder=False,
                                         categories=('non_responder',
                                                     'responder'),
                                         kind=<Kind.CATEGORICAL: 'categorical'>,
                                         drop_field='non_responder',
                                         column_names=('non_responder',
                                                       'responder'),
                                         colname_format='{name}[T.{field}]')],
             'treatment': [FactorMetadata(name='treatment',
                                          reduced_rank=True,
                                          custom_encoder=False,
                                          categories=('drugA', 'drugB'),
                          

In [9]:
variables_to_factors

defaultdict(set, {'treatment': {'treatment'}, 'response': {'response'}})

In a slightly more complex (and argueably useless) example, the result would look like this: 

In [10]:
factor_storage, variables_to_factors, materializer_class = formulaic_contrasts.get_factor_storage_and_materializer()
design_mat = materializer_class(df, record_factor_metadata=True).get_model_matrix(
    "~ np.log(biomarker) + C(treatment, contr.treatment(base='drugB')) * C(response)"
)

/home/sturm/projects/scverse/formulaic-contrasts/.venv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [11]:
pprint(factor_storage)

defaultdict(<class 'list'>,
            {'C(response)': [FactorMetadata(name='C(response)',
                                            reduced_rank=True,
                                            custom_encoder=True,
                                            categories=('non_responder',
                                                        'responder'),
                                            kind=<Kind.CATEGORICAL: 'categorical'>,
                                            drop_field=None,
                                            column_names=('responder',),
                                            colname_format='{name}[T.{field}]')],
             "C(treatment, contr.treatment(base='drugB'))": [FactorMetadata(name='C(treatment, '
                                                                                 "contr.treatment(base='drugB'))",
                                                                            reduced_rank=True,
                              

In [12]:
variables_to_factors

defaultdict(set,
            {'biomarker': {'np.log(biomarker)'},
             'np.log': {'np.log(biomarker)'},
             'C': {'C(response)',
              "C(treatment, contr.treatment(base='drugB'))"},
             'contr.treatment': {"C(treatment, contr.treatment(base='drugB'))"},
             'treatment': {"C(treatment, contr.treatment(base='drugB'))"},
             'response': {'C(response)'}})